# Contains Duplicate
Given an integer array nums, return true if any value appears more than once in the array, otherwise return false.

## problem/问题
### input 输入
nums：integer array
长度 n>= 0
### output 
bool 
是否存在i $\neq$ j,使得 nums[i] = nums[j]
### 本质
判断一个集合中是否存在 “元素冲突”
这是一个
- 状态检测问题（state detection）
- 非优化问题
- 非路径问题
- 纯判定问题


## Model 模型
***数学模型***是：
- 对问题中变量及其约束关系的抽象表达，
- 它定义了系统允许存在的解空间或可行空间。

模型强调：
- 表示（representation）
- 结构（structure）
- 约束（constraint）
但不必包含时间。

***数学系统***是：
- 在给定状态空间和约束下，
- 描述状态如何随时间或过程演化的模型。

系统强调：
- state
- transition
- evolution
即：
model + time / process

***两者关系***
静态模型 ⊂ 数学模型
***动态模型*** = 数学系统
换句话说：
所有系统都是模型，
但不是所有模型都是系统。

> 数学模型定义变量及其约束关系，从而模型确定解空间；

> 数学系统是在该解空间上引入状态与演化规则，使模型在时间或过程维度上运行，系统决定“路径”。




### 3. 数学模型
定义
- 变量
- 约束
- 解空间
#### 变量
- 序列 $X = (x_1,x_2, \dots, x_n) \in \mathbb{Z}^n$：表示数据流nums
- 集合 $ S $ :表示已观察到元素集合

#### 约束（核心）
唯一性约束：
$$
\forall i \neq j \\
x_i \neq x_j
$$
等价于
$|\{x_1,x_2,\dots, x_n\}| = n $

#### 解空间
合法输入集合
$ \mathcal{V} = \{X|所有元素互不相同\} $
非法输入集合
$ \mathcal{U} = \{X|有重复元素\}$

#### 判定函数（数学形式）
$$f(X) = 1 \quad if \quad X \in \mathcal{U}, 0 \quad if \quad X \in \mathcal{V}$$
这是一个布尔函数模型

### 数学系统（system）
系统 = 模型 + 状态 + 演化

#### 状态空间
1. 定义状态：
${S_k}$ = 已扫描元素集合
初始： $S_0 = \emptyset $

2. 状态转移（我的问题：xk 属于状态的一部分，还是策略）
读取第k个元素 $x_k$,状态更新：
若$x_k \notin S_k \rightarrow S_{k+1} = S_k \cup \{x_k \}$ 
若$x_k \in S_k \rightarrow violation \rightarrow $ 系统进入终止状态

3. 不变量（invariant）
$$ 
\forall k \\
S_k = unique(x_1,\dots,x_{k-1})
$$,即：
状态始终等于前缀唯一集合

4. 终止条件
系统停止当： $\exists k$, 使得：$x_k \in S_k$ 或： 扫描结束。




## 系统本质类型
这是一个： streaming constraint-monitoring system
特征：
- 输入： 数据流
- 状态： 累积集合
- 目标： 检测约束违反
- 行为： 顺序更新

## 算法系统（computational system）

### 5.1 数据结构
state ：
$ S \rightarrow $hash set
原因： membership 查询O(1)

### 5.2 计算演化
for k = 1 ... n:
    检查$x_k \in S$
    若是$\rightarrow $ return true
    否则 $\rightarrow$ 插入

### 复杂度
时间：O(n)
空间：O(n)



## 6 系统轨迹（trajectory）
系统运行产生轨迹：
$$ S_0 \rightarrow S_1 \rightarrow S_2 \rightarrow \dots \rightarrow S_k$$

## 7 用“优化系统” 重新介绍，将“约束系统”映射到“目标系统”
可重写为 违反度函数：
$$ 
L_k = I(x_k \in S_k)
$$
系统目标：minimize violation，只要$L_k = 1$系统即终止

7.1 系统类别
- 动态系统（state + transition）
    - policy 退化为 “environment dynamics / transition rule”
    - planner 只是推进一步一步演化，例：$x_{t+1} = f(x_t)$
- 规则系统（state + rule/policy）
- 监控系统（state + violation handler）
    - policy = “发现 violation 时怎么处理”
    - planner = 扫描/事件循环
    - state = 累积统计/已见集合
    - 例：contains duplicate
- 决策系统（state + policy + evaluation/cost/reward/utility）
    - policy 真正决定 action（π(s)）
    - planner 处理搜索/仿真/评估（MCTS、rollout、beam search…）
    - state 表示环境与 belief

工程上都可以用状态机模版：

- 对监控系统：Policy 可以叫 ViolationHandler
- 对动力系统：Policy 可以叫 Dynamics
- 对决策系统：才叫 Policy

### 7.3 Decision System 工程模版
五层
#### State 系统当前信息
- 数据
- 历史
- 内部变量
- 不变量

#### Policy
给定state + 输入 -》决定行为

#### Planner 推进系统运行
- 顺序扫描
- event loop
- simulation
- search

#### evaluator （learning evaluator）
评价objective / cost / reward
- 当前行为好不好
- 当前轨迹好不好
- 当前 state 是否健康

policy 其实不是系统核心。真正决定系统演化的是：***evaluator***。
因为：
- policy 只是动作规则
- evaluator 定义“好坏标准”

learning evaluator 的三条主线：
- supervised f(state, action) → score
    数据：
label
examples
典型：
CV classification
regression
anomaly detection

- RL （决策型）reward function / value
系统自己探索。

典型：
planning
control
agent

- Robustness / risk 学习：worst-case score
关注：
distribution shift
adversarial
uncertainty

#### observer
记录系统运行：
- logging
- metrics
- tracing
- monitoring
没有 observer：
- 系统不可诊断
- 不可调参
- 不可鲁棒化


## 8 工程系统（OOP/组件）
用python实现系统内对象，选择数据结构来保证不变量， 实现算法
### 8.1 核心角色（objects）
- DuplicateChecker(orchestrator/facade)
- SeenSet(state holder/value boundary)
- StreamCursor(迭代游标/数据源适配器)

### 结构关系（UML）
- DuplicateChecker has-a SeenSet（composition）
- DuplicateChecker uses-a StreamCursor 或 Iterable[int]（dependency）
- SeenSet 内部封装 set[int]（encapsulation）



In [1]:
#code
#todo：finish the algorithm using oop to express the relation(static,and dynamic) between the 

# State：SeenSet（维护“已见集合”的语义不变量）

# Policy：DuplicatePolicy（定义遇到重复时的决策 + 是否继续）

# Planner：ScanPlanner（驱动扫描流程，调用 policy）

# System/Facade：DuplicateSystem（组装并对外提供 run）

from __future__ import annotations
from dataclasses import dataclass, field
from typing import Iterable, Protocol


# ---------- State ----------
@dataclass
class SeenSet:
    """State holder. Invariant: _seen contains all observed values so far."""
    _seen: set[int] = field(default_factory=set)

    def contains(self, x: int) -> bool:
        return x in self._seen

    def add(self, x: int) -> None:
        self._seen.add(x)


# ---------- Policy ----------
class DuplicatePolicy(Protocol):
    """
    Policy decides what to do when we see a value.
    Return True to continue scanning, False to stop.
    """
    def on_new(self, x: int, state: SeenSet) -> bool: ...
    def on_duplicate(self, x: int, state: SeenSet) -> bool: ...
    def result(self) -> bool: ...


@dataclass
class StopOnFirstDuplicate:
    """Classic LeetCode behavior: stop immediately when any duplicate appears."""
    _found: bool = False

    def on_new(self, x: int, state: SeenSet) -> bool:
        state.add(x)
        return True  # continue

    def on_duplicate(self, x: int, state: SeenSet) -> bool:
        self._found = True
        return False  # stop

    def result(self) -> bool:
        return self._found


# ---------- Planner ----------
@dataclass
class ScanPlanner:
    """Drives the trajectory over the stream until policy says stop."""
    def run(self, nums: Iterable[int], state: SeenSet, policy: DuplicatePolicy) -> bool:
        for x in nums:
            if state.contains(x):
                if not policy.on_duplicate(x, state):
                    return policy.result()
            else:
                if not policy.on_new(x, state):
                    return policy.result()
        return policy.result()


# ---------- System / Facade ----------
@dataclass
class DuplicateSystem:
    planner: ScanPlanner = field(default_factory=ScanPlanner)

    def run(self, nums: Iterable[int], policy: DuplicatePolicy | None = None) -> bool:
        state = SeenSet()
        policy = policy or StopOnFirstDuplicate()
        return self.planner.run(nums, state, policy)


In [ ]:
from dataclasses import dataclass, field


@dataclass
class CollectAllDuplicates:
    """Do not stop. Collect every duplicate observed."""
    duplicates: set[int] = field(default_factory=set)

    def on_new(self, x: int, state: SeenSet) -> bool:
        state.add(x)
        return True  # continue scanning

    def on_duplicate(self, x: int, state: SeenSet) -> bool:
        self.duplicates.add(x)
        return True  # still continue

    def result(self) -> bool:
        return len(self.duplicates) > 0

    def report(self) -> set[int]:
        return self.duplicates


In [ ]:
#evaluator
@dataclass
class DuplicateEvaluator:
    duplicates: int = 0

    def on_duplicate(self):
        self.duplicates += 1

    def score(self) -> int:
        return self.duplicates


In [ ]:
#observor
@dataclass
class SystemObserver:
    events: list[str] = field(default_factory=list)

    def log(self, msg: str):
        self.events.append(msg)


In [ ]:
# reflector planner
def run(nums, state, policy, evaluator, observer):
    for x in nums:
        observer.log(f"Read {x}")

        if state.contains(x):
            observer.log(f"Duplicate {x}")
            evaluator.on_duplicate()

            if not policy.on_duplicate(x, state):
                break
        else:
            state.add(x)
            policy.on_new(x, state)

    return evaluator.score()


## 从工程系统进入“Learning Decision System”（学习型决策系统）
ML/RL/Robustness 的核心范式

### 系统结构设计
之前：state → policy → planner → evaluator → observer

现在：

state → policy → planner
         ↓
      learned evaluator
         ↓
       update model/ training
         ↓
影响下一次 policy / planner

系统出现：***闭环学习***（learning control loop）

### evaluator 学习意味着什么

它不再是：
- 手写规则：
- 重复数
- 违规
- cost

而是：
- 从数据中学：
- 风险
- 相似度
- anomaly score
- reward